In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [3]:
from dask import distributed as dds
from dask import array as da
import time
import math
import numpy as np

# Connecting to existing cluster

We need to know cluster's scheduler's ip address and port

In [4]:
#cluster = dds.LocalCluster()
#client = dds.Client(cluster)
client = dds.Client('tcp://127.0.0.1:51001') # always check port
client

Client Scheduler: tcp://127.0.0.1:51001 Dashboard: http://127.0.0.1:8787,Cluster Workers: 4 Cores: 4 Memory: 8.49 GB


# Array creation & elementwise computation

In [5]:
MAT_SIZE=(20000, 20000)

**But how big is this?**
```
    20 000 x 20 000 numbers = 400 000 000 numbers

    each number is 8 Bytes (int64 or float64 are defaults for numpy)

    400 000 000 x 8 Bytes = 3 200 000 000 Bytes
    3 200 000 000 / 1024 = 3 125 000 KBytes
    3 125 000 / 1024 = 3 051.75 MBytes
    3 051.75 / 1024 = 2.98 GB
```
**2.98 GB** - pretty much... 

## numpy

In [6]:
%%time
xnp = np.random.randint(0, 10000, size=MAT_SIZE)

Wall time: 7.44 s


In [7]:
%%time
ynp = np.sin(xnp) ** 2 + np.cos(xnp) ** 2

KeyboardInterrupt: 

## Dask

In [9]:
%%time
x = da.random.randint(0, 10000, size=MAT_SIZE, chunks=(2500, 2500))
x = x.persist()
dds.wait(x)

Wall time: 4.06 s


In [10]:
%%time
y = da.sin(x) ** 2 + da.cos(x) ** 2

Wall time: 15.6 ms


If you don't need to get the numpy array explicitly (e.g. it is only an intermediate computation step) - you can just __persist__ it and wait till it's evaluated.

Collecting such big structure to single numpy array can be time-consuming.

In [11]:
%%time
y = y.persist()
dds.wait(y)

Wall time: 13.9 s


In [14]:
%%time
y[:10000,:10000].compute()

Wall time: 21.8 s


array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

**Some other computations**

In [22]:
%%time
y.transpose()

Wall time: 0 ns


dask.array<transpose, shape=(20000, 20000), dtype=float64, chunksize=(2500, 2500)>

In [26]:
!mkdir data\numpy_data

In [27]:
%%time
da.to_npy_stack('data/numpy_data',y, axis=0)

Wall time: 36.4 s


distributed.client - WARNING - Client report stream closed to scheduler
tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFAD63F28>, <tornado.concurrent.Future object at 0x0000027BFB32C860>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\tcp.py", line 174, in read
    n_frames = yield stream.read_bytes(8)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\iostream.py", line 324, in read_bytes
    self._try_inline_read()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\iostream.py", line 709, in _try_inline_read
    self._check_closed()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\iostream.py", line 925, in _check_closed
    raise StreamClosedError(real_error=self.error)
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent c

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFAD6CF28>, <tornado.concurrent.Future object at 0x0000027BFC327978>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\tcp.py", line 174, in read
    n_frames = yield stream.read_bytes(8)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\iostream.py", line 324, in read_bytes
    self._try_inline_read()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\iostream.py", line 709, in _try_inline_read
    self._check_closed()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\iostream.py", line 925, in _check_closed
    raise StreamClosedError(real_error=self.error)
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distri

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFC249E18>, <tornado.concurrent.Future object at 0x0000027BFC3270B8>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFC249EA0>, <tornado.concurrent.Future object at 0x0000027BFB03C4E0>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex

...

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFC4F77B8>, <tornado.concurrent.Future object at 0x0000027BFC351860>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFC40A400>, <tornado.concurrent.Future object at 0x0000027BFC524358>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFC510A60>, <tornado.concurrent.Future object at 0x0000027BFC17F278>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFB25D510>, <tornado.concurrent.Future object at 0x0000027BFC3272E8>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x0000027BFC549EA0>, <tornado.concurrent.Future object at 0x0000027BFC5467F0>)
Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\distributed\comm\core.py", line 185, in connect
    quiet_exceptions=EnvironmentError)
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\gen.py", line 1055, in run
    value = future.result()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\concurrent.py", line 238, in result
    raise_exc_info(self._exc_info)
  File "<string>", line 4, in raise_exc_info
tornado.gen.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\ioloop.py", line 605, in _run_callback
    ret = callback()
  File "C:\Users\mkosturek\Anaconda3\lib\site-packages\tornado\stack_contex